# DELAY VERIFICATION

In [3]:
import pandas as pd
import datetime
from datetime import datetime
from tqdm import tqdm

## Import the data

In [48]:
data=pd.read_csv("delays.csv")

## Data cleaning

In [7]:
data.dtypes

datetime      object
id            object
carrier       object
date          object
connection    object
arrival       object
delay         object
name          object
dtype: object

Convert date in datetime

In [8]:
data["datetime"]=pd.to_datetime(data["datetime"])
data["date"]=pd.to_datetime(data["date"])
data.dtypes

datetime      datetime64[ns]
id                    object
carrier               object
date          datetime64[ns]
connection            object
arrival               object
delay                 object
name                  object
dtype: object

Convert delay in integer value (number of minutes)

In [9]:
data["delay_old"]=data["delay"]

data["delay"]=data["delay"].apply(lambda x: x.replace(' min', ''))
data["delay"]=pd.to_numeric(data["delay"])
data.dtypes

datetime      datetime64[ns]
id                    object
carrier               object
date          datetime64[ns]
connection            object
arrival               object
delay                  int64
name                  object
delay_old             object
dtype: object

ID correction

In [10]:
data["id_old"]=data["id"]

data["id"]=data["id_old"].apply(lambda x:' '.join(x.split()))
data.head()

,datetime,id,carrier,date,connection,arrival,delay,name,delay_old,id_old
0,2022-05-16,41002/3 (102) POLONIA,PKP Intercity,2022-05-15,Bohumin - Warszawa Wschodnia,21:31,162,Warszawa Zachodnia,162 min,41002/3 (102) POLONIA
1,2022-05-16,91888/9 (102),Koleje Mazowieckie,2022-05-16,Siedlce - Warszawa Zachodnia,00:15,0,Warszawa Zachodnia,0 min,91888/9 (102)
2,2022-05-16,21644/5 (102),Koleje Mazowieckie,2022-05-16,Radom Główny - Warszawa Wschodnia,00:29,0,Warszawa Zachodnia,0 min,21644/5 (102)
3,2022-05-16,19300/1 (102),Koleje Mazowieckie,2022-05-16,Warszawa Wschodnia - Grodzisk Mazowiecki,00:34,0,Warszawa Zachodnia,0 min,19300/1 (102)
4,2022-05-16,38170/1 (102) USTRONIE,PKP Intercity,2022-05-16,Kraków Główny - Kołobrzeg,00:55,0,Warszawa Zachodnia,0 min,38170/1 (102) USTRONIE


New ID creation: 1 ID per travel

In [11]:
def new_id_pattern(date, id):
    return id+"_"+str(date.day)+'-'+str(date.month)+'-'+str(date.year)+'_'+str(date.hour)+'-'+str(date.minute)


id_list=data["id"].unique()
id_list

new_data=pd.DataFrame(columns=data.columns)
new_data["new_id"]=pd.Series()

for id in tqdm(id_list):
        
    #prepare dataset columns and re-index
    data_id=data[data['id']==id].copy().reset_index()
    data_id["new_id"]=''

    #initialize first line: ind=0
    date_dep=data_id.iloc[0]['datetime']
    data_id.loc[0,'new_id']=new_id_pattern(date_dep, id)

    #loop parameters
    ind=1 #used to loop over the data
    #stop condition of the loop
    if ind==data_id.shape[0]:stop=True
    else:stop=False 

    #loop to create new_ind value for the current ind
    while not stop:
        diff_date=data_id.loc[ind,'datetime']-data_id.loc[ind-1,'datetime'] #date diff between current and previous ind
        
        #case of 2 successives datetime report: same travel => SAME ID
        if diff_date.total_seconds() <=5*60: #multiplied by 60 as we compare in seconds
            data_id.loc[ind,"new_id"]=new_id_pattern(date_dep,id) #set new_ind value for current ind
            ind+=1

        #case of another travel=> ANOTHER ID
        else: 
            date_dep=date_dep=data_id.loc[ind,'datetime'] #set the departure date for the new travel: will be overwritten for each new travel
            data_id.loc[ind,"new_id"]=new_id_pattern(date_dep,id) #set new_ind value for current ind
            ind+=1
        
        #Stop condition
        if ind==data_id.shape[0]:
            stop=True

    new_data=pd.concat([new_data, data_id], ignore_index = True)
    

C:\Users\louis\AppData\Local\Temp\ipykernel_8816\822593433.py:9: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_data["new_id"]=pd.Series()
  0%|          | 13/7947 [00:05<56:45,  2.33it/s]  


KeyboardInterrupt: 

In [72]:
new_data.to_csv("new_data.csv")

## Features creation

In [61]:
data=pd.read_csv("new_data.csv",index_col='index')
data["datetime"]=pd.to_datetime(data["datetime"])
data["date"]=pd.to_datetime(data["date"])
data=data.drop(columns=["Unnamed: 0"])
data

,datetime,id,carrier,date,connection,arrival,delay,name,delay_old,id_old,final_delay
index,,,,,,,,,,,
0,2022-05-16 00:00:00,41002/3 (102) POLONIA_16-5-2022_0-0,PKP Intercity,2022-05-15,Bohumin - Warszawa Wschodnia,21:31,162,Warszawa Zachodnia,162 min,41002/3 (102) POLONIA,NaN
1,2022-05-16 00:00:00,91888/9 (102)_16-5-2022_0-0,Koleje Mazowieckie,2022-05-16,Siedlce - Warszawa Zachodnia,00:15,0,Warszawa Zachodnia,0 min,91888/9 (102),NaN
2,2022-05-16 00:00:00,21644/5 (102)_16-5-2022_0-0,Koleje Mazowieckie,2022-05-16,Radom Główny - Warszawa Wschodnia,00:29,0,Warszawa Zachodnia,0 min,21644/5 (102),NaN
3,2022-05-16 00:00:00,19300/1 (102)_16-5-2022_0-0,Koleje Mazowieckie,2022-05-16,Warszawa Wschodnia - Grodzisk Mazowiecki,00:34,0,Warszawa Zachodnia,0 min,19300/1 (102),NaN
4,2022-05-16 00:00:00,38170/1 (102) USTRONIE_16-5-2022_0-0,PKP Intercity,2022-05-16,Kraków Główny - Kołobrzeg,00:55,0,Warszawa Zachodnia,0 min,38170/1 (102) USTRONIE,NaN
...,...,...,...,...,...,...,...,...,...,...,...
3718165,2022-05-29 23:55:00,53172/3 ROZEWIE_29-5-2022_21-35,PKP Intercity,2022-05-30,Gdynia Główna - Kraków Główny,07:25,4,Mysłowice,4 min,53172/3 ROZEWIE,NaN
3718166,2022-05-29 23:55:00,15417_29-5-2022_23-15,Koleje Mazowieckie,2022-05-30,Warszawa Zachodnia peron 9 - Mława,00:23,0,Nowy Dwór Mazowiecki,0 min,15417,NaN
3718167,2022-05-29 23:55:00,38170/1 USTRONIE_29-5-2022_19-35,PKP Intercity,2022-05-30,Kraków Główny - Kołobrzeg,02:16,0,Nowy Dwór Mazowiecki,0 min,38170/1 USTRONIE,NaN


In [64]:
dd=data.loc[data.groupby('id')["delay"].idxmax()]
dd[dd["delay"]>0]

,datetime,id,carrier,date,connection,arrival,delay,name,delay_old,id_old,final_delay
index,,,,,,,,,,,
1065463,2022-05-19 19:55:00,10102/3 (130) ZAMENHOF_19-5-2022_19-55,PKP Intercity,2022-05-19,Warszawa Zachodnia - Białystok,20:18,23,Warszawa Centralna,23 min,10102/3 (130) ZAMENHOF,NaN
1830753,2022-05-22 19:55:00,10102/3 (130) ZAMENHOF_22-5-2022_19-55,PKP Intercity,2022-05-22,Warszawa Zachodnia - Białystok,20:18,7,Warszawa Centralna,7 min,10102/3 (130) ZAMENHOF,NaN
240841,2022-05-16 20:00:00,10102/3 (49) ZAMENHOF_16-5-2022_20-0,PKP Intercity,2022-05-16,Warszawa Zachodnia - Białystok,20:30,7,Warszawa Wschodnia,7 min,10102/3 (49) ZAMENHOF,NaN
1589499,2022-05-21 19:35:00,10102/3 (49) ZAMENHOF_21-5-2022_19-35,PKP Intercity,2022-05-21,Warszawa Zachodnia - Białystok,20:18,5,Warszawa Centralna,5 min,10102/3 (49) ZAMENHOF,NaN
1831704,2022-05-22 20:00:00,10102/3 (49) ZAMENHOF_22-5-2022_20-0,PKP Intercity,2022-05-22,Warszawa Zachodnia - Białystok,20:30,7,Warszawa Wschodnia,7 min,10102/3 (49) ZAMENHOF,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1096201,2022-05-19 22:55:00,99972/3_19-5-2022_22-15,SKM Warszawa,2022-05-19,Sulejówek Miłosna - Warszawa Lotnisko Chopina,23:02,1,Warszawa Ochota,1 min,99972/3,NaN
1382156,2022-05-20 23:35:00,99972/3_20-5-2022_23-35,SKM Warszawa,2022-05-20,Sulejówek Miłosna - Warszawa Lotnisko Chopina,23:20,18,Warszawa Służewiec,18 min,99972/3,NaN
547714,2022-05-17 23:35:00,99990/1 (117)_17-5-2022_22-55,SKM Warszawa,2022-05-17,Sulejówek Miłosna - Warszawa Zachodnia,23:26,18,Warszawa Wschodnia,18 min,99990/1 (117),NaN


In [62]:
data[data["id"]=="10102/3 (130) ZAMENHOF_19-5-2022_19-55"]

,datetime,id,carrier,date,connection,arrival,delay,name,delay_old,id_old,final_delay
index,,,,,,,,,,,
1065463,2022-05-19 19:55:00,10102/3 (130) ZAMENHOF_19-5-2022_19-55,PKP Intercity,2022-05-19,Warszawa Zachodnia - Białystok,20:18,23,Warszawa Centralna,23 min,10102/3 (130) ZAMENHOF,NaN


Target value: final value of delay

In [68]:
dd=data[data['id_old']=="10102/3 (130) ZAMENHOF"]
dd[dd['date']=='2022-05-17']

,datetime,id,carrier,date,connection,arrival,delay,name,delay_old,id_old,final_delay
index,,,,,,,,,,,
513901,2022-05-17 19:55:00,10102/3 (130) ZAMENHOF_17-5-2022_19-55,PKP Intercity,2022-05-17,Warszawa Zachodnia - Białystok,20:18,0,Warszawa Centralna,0 min,10102/3 (130) ZAMENHOF,NaN
516102,2022-05-17 20:05:00,10102/3 (130) ZAMENHOF_17-5-2022_20-5,PKP Intercity,2022-05-17,Warszawa Zachodnia - Białystok,20:30,0,Warszawa Wschodnia,0 min,10102/3 (130) ZAMENHOF,NaN


Number of delays a day

In [22]:

id=id_list[0]
data[data["id"]==id].reset_index().loc[data[data["id"]==id].shape[0], 'delay']

KeyError: 19

In [60]:
data[data["id"]=="31100/1 (102) MALINOWSKI"]

,datetime,id,carrier,date,connection,arrival,delay,name,delay_old,id_old
45,2022-05-16,31100/1 (102) MALINOWSKI,PKP Intercity,2022-05-15,Przemyśl Główny - Warszawa Wschodnia,23:31,34,Warszawa Wschodnia,34 min,31100/1 (102) MALINOWSKI


In [47]:
print(data["id"].iloc[3718078])
' '.join(data["id"].iloc[3718078].split())
r

53172/3  ROZEWIE


'53172/3 ROZEWIE'